# This is the first notebook (preprocessing data)

In [ ]:
import os
import json
from tqdm.auto import tqdm
import numpy as np

import sys
sys.path.append('../src/')

import utils.utils as utils
import utils.process_utils as process_utils

import matplotlib.pyplot as plt

from pykalman import KalmanFilter
%load_ext autoreload
%autoreload 2
np.random.seed(7)

## Step 1: Transform Data to Numpy Array (3 hours)

In [ ]:
DATAPATH = None # Replace with your training data root
path_list = [] 

# find all data paths
for root, dirs, files in os.walk(os.path.abspath(DATAPATH)):
    files.sort()
    for file in files:
        path_list.append(os.path.join(root, file))

In [ ]:
object_type = 'vehicle' # Replace with vehicle, pedestrian or cyclist

object_type_id = None
if object_type == 'vehicle':
    object_type_id = 1
elif object_type == 'pedestrian': 
    object_type_id = 2
elif object_type == 'cyclist':
    object_type_id = 3
    
agt_save_path = 'data/' + object_type + '/agt_trajs_json/'
ego_save_path = 'data/ego/ego_trajs_json/'
if not os.path.exists(agt_save_path):
    os.makedirs(agt_save_path)
if not os.path.exists(ego_save_path):
    os.makedirs(ego_save_path)

i_e, i_a = 0, 0
for k in tqdm(path_list):
    i_ego_sub, i_agt_sub = process_utils.process_raw_data(k, 
                                                          object_type_id = object_type_id,
                                                          ego_save_path = ego_save_path,
                                                          agt_save_path = agt_save_path,
                                                          save_ego = True,
                                                          save_agt = True) 
    i_e = i_e + i_ego_sub
    i_a = i_a + i_agt_sub
    
print('finish')
print('number of ego trajs:', i_e, 'number of agt trajs:', i_a)

## Step 2: Preprocessing Ego Trajectories (Smoothing)

In [ ]:
A_LON_MAX, A_LON_MIN = 6, -9.8 #[m/s2], acceleration not in this range will be considered as infeasible
DELTA_POS_TOL = 2 #[m], greater distance between measurement and smoothed trajectory will be considered as outlier
LEAST_MOVING_D = 0.5 #[m], less distance between start and end points will be considered as not moving object 
DELTA_T_TOL = 0.5 #[s], trajectory with greater time difference will not be interested 

In [ ]:
ego_path_list = []

for root, dirs, files in os.walk(os.path.abspath('data/ego/ego_trajs_json/')):
    files.sort()
    for file in files:
        ego_path_list.append(os.path.join(root, file))

In [ ]:
origin_num_points_in_traj = 91 # We have 91 sample points in one 9s trajectory
num_points_in_trajs = [91, 81, 71, 61, 51,41,31,21,11] # Target Number of points after splitting

for num_points_in_traj in num_points_in_trajs:
    print('Processing sub-trajectory with {} points'.format(num_points_in_traj))
    np.random.seed(7)
    posterior_means = np.zeros((len(path_list), num_points_in_traj, 6))
    posterior_a_lon = np.zeros((len(path_list), num_points_in_traj))

    outlier_idx_acc = []
    outlier_idx_time = []
    outlier_idx_pos = []
    outlier_idx_not_moving = []
    outlier_idx = []
    start_point_indicies = []
    
    
    # Initializing Kalman Smoother Parameters
    R = np.diag([0.005, 0.005, 0.1, 0.1]) # Observation Cov
    initial_state_mean = np.zeros(6)
    initial_state_covariance = np.diag(np.array([R[0, 0], R[2, 2], 10, R[1, 1], R[3, 3], 10]))

    dt = 0.1

    observation_matrix = np.array([[1, 0, 0, 0, 0, 0],
                                   [0, 0, 0, 1, 0, 0],
                                   [0, 1, 0, 0, 0, 0],
                                   [0, 0, 0, 0, 1, 0]])

    Q = np.kron(np.eye(2), np.array([[(dt**5)/20, (dt**4)/8, (dt**3)/6],
                          [(dt**4)/8,  (dt**3)/3, (dt**2)/2],
                          [(dt**3)/6,  (dt**2)/2, dt]]).astype(np.float32)) # Process Cov
    
    parameter = {'R': np.diag(R), 'init_cov':np.diag(initial_state_covariance)}

    for idx, path in enumerate(tqdm(ego_path_list)):
        T, traj, start_point_idx = utils.extract_traj_data(file_path=path, target_num_of_points=num_points_in_traj , traj_type = 'ego_traj')
        start_point_indicies.append(start_point_idx)

        dT = T[1:] - T[:-1]

        transition_matrices = [np.kron(np.eye(2), np.array([[1, dt, 0.5* (dt**2)], 
                                                             [0, 1, dt],
                                                             [0, 0, 1]])) for dt in dT]

        initial_state_mean[1] = traj[0,2] # vx
        initial_state_mean[4] = traj[0,3] # vy

        # We smooth the measurement
        kf = KalmanFilter(transition_matrices = transition_matrices, observation_matrices = observation_matrix, transition_covariance = Q,
                      observation_covariance = R, initial_state_mean = initial_state_mean, initial_state_covariance = initial_state_covariance)

        smoothed_state_means, smoothed_state_covariances  = kf.smooth(traj)

        v_xy = smoothed_state_means[:, [1, 4]]
        a_xy = smoothed_state_means[:, [2, 5]]
        
        # Calculate longitudinal acceleration based on velocity vector
        a_lon = np.array([np.dot(a ,v) / np.linalg.norm(v) for a, v in zip(a_xy.reshape(-1,2), v_xy.reshape(-1,2))])
        
        a_lon = a_lon.reshape(-1, num_points_in_traj)

        posterior_means[idx] = smoothed_state_means
        posterior_a_lon[idx] = a_lon
        
        # Find outlier with infeasible acceleration and deceleration
        outlier_points_acc = np.argwhere((a_lon > A_LON_MAX) |(a_lon < A_LON_MIN))
        if outlier_points_acc.shape[0] > 0:
            outlier_idx_acc.append(idx)
        
        # Find outlier with high deviation between ks outout and measurement
        delta_pos_max = np.amax(np.linalg.norm(traj[:,[0,1]] - smoothed_state_means[:, [0,3]], axis=1))
        
        if delta_pos_max > DELTA_POS_TOL:
            outlier_idx_pos.append(idx)
            
        # Find not moving outlier    
        moving_d = np.linalg.norm(traj[-1, :2])
        
        if moving_d <= LEAST_MOVING_D:
            outlier_idx_not_moving.append(idx)

        # Find outlier with time issue
        if (T[-1] >= ((num_points_in_traj-1) / 10) + DELTA_T_TOL ) | (T[-1] < ((num_points_in_traj-1) / 10) - DELTA_T_TOL):
            outlier_idx_time.append(idx)



    outlier_idx = set(outlier_idx_acc) | set(outlier_idx_time) | set(outlier_idx_pos) | set(outlier_idx_not_moving)
    print('Find {0} outliers, from which {1} acc outlier, {2} time outlier, {3} pos outlier, {4} not moving'.format(len(outlier_idx), len(outlier_idx_acc), len(outlier_idx_time), len(outlier_idx_pos),len(outlier_idx_not_moving)))

    
    PATH = "data/ego/ego_trajs_" + str(num_points_in_traj) + "_json"
    if not os.path.exists(PATH):
        os.makedirs(PATH)

    with open(PATH + "/ego_trajs_outlier_indicies.json", "w") as write_file:
        json.dump(list(outlier_idx), write_file, cls=utils.NumpyEncoder)

    with open(PATH + "/ego_trajs_outlier_indicies_pos.json", "w") as write_file:
        json.dump(list(outlier_idx_pos), write_file, cls=utils.NumpyEncoder)

    with open(PATH + "/ego_trajs_outlier_indicies_acc.json", "w") as write_file:
        json.dump(outlier_idx_acc, write_file, cls=utils.NumpyEncoder)

    with open(PATH + "/ego_trajs_outlier_indicies_time.json", "w") as write_file:
        json.dump(outlier_idx_time, write_file, cls=utils.NumpyEncoder)
        
    with open(PATH + "/ego_trajs_outlier_indicies_not_moving.json", "w") as write_file:
        json.dump(outlier_idx_not_moving, write_file, cls=utils.NumpyEncoder)

    with open(PATH + "/ego_trajs_start_point_indicies.json", "w") as write_file:
        json.dump(start_point_indicies, write_file, cls=utils.NumpyEncoder)

    with open(PATH + "/ego_trajs_filtered_CA.json", "w") as write_file:
        json.dump(posterior_means, write_file, cls=utils.NumpyEncoder)

    with open(PATH + "/ego_trajs_filtered_a_lon.json", "w") as write_file:
        json.dump(posterior_a_lon, write_file, cls=utils.NumpyEncoder)
        
    with open(PATH + "/ks_parameter.json", "w") as write_file:
        json.dump(parameter, write_file, cls=utils.NumpyEncoder)

# Preprocessing Agent Trajectories (Smoothing, 6 hours)

In [ ]:
agt_file_path_list = []
np.random.seed(7)
object_type = 'vehicle'

select_file_idx = np.sort(np.random.choice(range(1829161), size=600000, replace=False)) # We randomly choose 60k data from all 1829161 agent trajectories (vehicle) for further processing

for root, dirs, files in os.walk(os.path.abspath('data/'+ object_type + '/agt_trajs_json/')):
    files.sort()
    for idx, file in enumerate(tqdm(files)):
        if '.json' not in file or 'checkpoint' in file: # ignore unrelated file
            continue
        
        # When processing vehicle trajs, we only choose 600000 for further processing, else we use all trajs.
        if idx in select_file_idx and object_type=='vehicle':
            agt_file_path_list.append(os.path.join(root, file))
        else:
            agt_file_path_list.append(os.path.join(root, file))

len(agt_file_path_list)

In [ ]:
A_LON_MAX, A_LON_MIN = None, None #[m/s2], acceleration not in this range will be considered as infeasible
if object_type =='vechile':
    A_LON_MAX, A_LON_MIN = 6, -9.8 
elif object_type == 'pedestrian':
    A_LON_MAX, A_LON_MIN = 2, -3
elif object_type == 'cyclist':
    A_LON_MAX, A_LON_MIN = 2, -4
    
DELTA_POS_TOL = 2 #[m], greater distance between measurement and smoothed trajectory will be considered as outlier
LEAST_MOVING_D = 0.5 #[m], less distance between start and end points will be considered as not moving object 
DELTA_T_TOL = 0.5 #[s], trajectory with greater time difference will not be interested 

In [ ]:
origin_num_points_in_traj = 91 # We have 91 sample points in one 9s trajectory
num_points_in_trajs = [91,81,71,61,51,41,31,21, 11] # Target Number of points after splitting

for num_points_in_traj in num_points_in_trajs:
    print('Processing sub-trajectory with {} points'.format(num_points_in_traj))
    np.random.seed(7)
    posterior_means = np.zeros((len(agt_file_path_list), num_points_in_traj, 6))
    posterior_a_lon = np.zeros((len(agt_file_path_list), num_points_in_traj))

    outlier_idx_acc = []
    outlier_idx_time = []
    outlier_idx_pos = []
    outlier_idx_not_moving = []
    outlier_idx = []
    start_point_indicies = []
    
    out_of_view = 0
    
    
    # Initializing Kalman Smoother Parameters
    R = np.diag([0.01, 0.01, 1, 1]) 

    initial_state_mean = np.zeros(6)
    initial_state_covariance = np.diag(np.array([R[0, 0], R[2, 2], 10, R[1, 1], R[3, 3], 10]))

    dt = 0.1

    observation_matrix = np.array([[1, 0, 0, 0, 0, 0],
                                   [0, 0, 0, 1, 0, 0],
                                   [0, 1, 0, 0, 0, 0],
                                   [0, 0, 0, 0, 1, 0]])
    
    Q = np.kron(np.eye(2), np.array([[(dt**5)/20, (dt**4)/8, (dt**3)/6],
                          [(dt**4)/8,  (dt**3)/3, (dt**2)/2],
                          [(dt**3)/6,  (dt**2)/2, dt]]).astype(np.float32)) # Process Cov
    
        
    parameter = {'R': np.diag(R), 'init_cov':np.diag(initial_state_covariance)}
    
    for idx, path in enumerate(tqdm(agt_file_path_list)):
        T, traj, start_point_idx = utils.extract_traj_data(file_path=path, target_num_of_points=num_points_in_traj , traj_type = 'agt_traj')
        
        start_point_indicies.append(start_point_idx)

        dT = T[1:] - T[:-1]

        transition_matrices = [np.kron(np.eye(2), np.array([[1, dt, 0.5* (dt**2)], 
                                                             [0, 1, dt],
                                                             [0, 0, 1]])) for dt in dT]

        initial_state_mean[1] = traj[0,2]
        initial_state_mean[4] = traj[0,3]


        kf = KalmanFilter(transition_matrices = transition_matrices, observation_matrices = observation_matrix, transition_covariance = Q,
                      observation_covariance = R, initial_state_mean = initial_state_mean, initial_state_covariance = initial_state_covariance)

        smoothed_state_means, smoothed_state_covariances  = kf.smooth(traj)

        v_xy = smoothed_state_means[:, [1, 4]]
        a_xy = smoothed_state_means[:, [2, 5]]

        a_lon = np.array([np.dot(a ,v) / np.linalg.norm(v) for a, v in zip(a_xy.reshape(-1,2), v_xy.reshape(-1,2))])

        a_lon = a_lon.reshape(-1, num_points_in_traj)

        posterior_means[idx] = smoothed_state_means
        posterior_a_lon[idx] = a_lon

        delta_pos_max = np.amax(np.linalg.norm(traj[:,[0,1]] - smoothed_state_means[:, [0,3]], axis=1))
        
        moving_d = np.linalg.norm(traj[-1, :2])
        

        if object_type == 'vehicle':
            outlier_points_acc = np.argwhere((a_lon > A_LON_MAX) |(a_lon < A_LON_MIN))
            
        if outlier_points_acc.shape[0] > 0:
            outlier_idx_acc.append(idx)
            
        if delta_pos_max > DELTA_POS_TOL:
            outlier_idx_pos.append(idx)
            
        if moving_d <= LEAST_MOVING_D:
            outlier_idx_not_moving.append(idx)
            
        if (T[-1] >= ((num_points_in_traj-1) / 10) + DELTA_T_TOL ) | (T[-1] < ((num_points_in_traj-1) / 10) - DELTA_T_TOL):
            outlier_idx_time.append(idx)

    outlier_idx = set(outlier_idx_acc) | set(outlier_idx_time) | set(outlier_idx_pos) | set(outlier_idx_not_moving)
    
    # we choose 300000 trajs without outlier for our experiments
    if object_type == 'vehicle':
        non_outlier_idx = list(set(range(idx+1)) - outlier_idx)
        non_outlier_idx = np.sort(np.random.choice(non_outlier_idx, size=300000, replace=False))
    

    print('Find {0} outliers, from which {1} acc outlier, {2} time outlier, {3} pos outlier, {4} not moving'.format(len(outlier_idx), len(outlier_idx_acc), len(outlier_idx_time), len(outlier_idx_pos),len(outlier_idx_not_moving)))
    
    
    PATH = "data/"+ object_type + "/agt_trajs_" + str(num_points_in_traj) + "_json"
    if not os.path.exists(PATH):
        os.makedirs(PATH)

    with open(PATH + "/agt_trajs_outlier_indicies.json", "w") as write_file:
        json.dump(list(outlier_idx), write_file, cls=utils.NumpyEncoder)

    with open(PATH + "/agt_trajs_outlier_indicies_pos.json", "w") as write_file:
        json.dump(list(outlier_idx_pos), write_file, cls=utils.NumpyEncoder)

    with open(PATH + "/agt_trajs_outlier_indicies_acc.json", "w") as write_file:
        json.dump(outlier_idx_acc, write_file, cls=utils.NumpyEncoder)

    with open(PATH + "/agt_trajs_outlier_indicies_time.json", "w") as write_file:
        json.dump(outlier_idx_time, write_file, cls=utils.NumpyEncoder)
        
    with open(PATH + "/agt_trajs_outlier_indicies_not_moving.json", "w") as write_file:
        json.dump(outlier_idx_not_moving, write_file, cls=utils.NumpyEncoder)
    
    if object_type == 'vehicle': # only store the 300000 selected data 
        with open(PATH + "/agt_trajs_select_non_outlier_indicies.json", "w") as write_file:
            json.dump(select_file_idx[non_outlier_idx], write_file, cls=utils.NumpyEncoder)

        with open(PATH + "/agt_trajs_start_point_indicies.json", "w") as write_file:
            json.dump(np.array(start_point_indicies)[non_outlier_idx], write_file, cls=utils.NumpyEncoder)

        with open(PATH + "/agt_trajs_filtered_CA.json", "w") as write_file:
            json.dump(posterior_means[non_outlier_idx], write_file, cls=utils.NumpyEncoder)
            
        with open(PATH + "/agt_trajs_filtered_a_lon.json", "w") as write_file:
            json.dump(posterior_a_lon[non_outlier_idx], write_file, cls=utils.NumpyEncoder)
    else: # store all data 
        with open(PATH + "/agt_trajs_start_point_indicies.json", "w") as write_file:
            json.dump(start_point_indicies, write_file, cls=utils.NumpyEncoder)

        with open(PATH + "/agt_trajs_filtered_CA.json", "w") as write_file:
            json.dump(posterior_means, write_file, cls=utils.NumpyEncoder)

        with open(PATH + "/agt_trajs_filtered_a_lon.json", "w") as write_file:
            json.dump(posterior_a_lon, write_file, cls=utils.NumpyEncoder)
        
    with open(PATH + "/ks_parameter.json", "w") as write_file:
        json.dump(parameter, write_file, cls=utils.NumpyEncoder)